I couldn't crawl glassdoor data in a normal way, like by BeautifulSoup. 
Glassdoor doesn't allow bots to crawl data, and access to the specified resource is forbidden.

I referred to "https://github.com/arapfaik/scraping-glassdoor-selenium/blob/master/glassdoor%20scraping.ipynb".

1. data sientist jobs in the US (using 30 urls)
url = 'https://www.glassdoor.com/Job/us-data-scientist-jobs-SRCH_IL.0,2_IN1_KO3,17.htm'
url = 'https://www.glassdoor.com/Job/us-data-scientist-jobs-SRCH_IL.0,2_IN1_KO3,17_IP2.htm'
:
url = 'https://www.glassdoor.com/Job/us-data-scientist-jobs-SRCH_IL.0,2_IN1_KO3,17_IP30.htm'

2. software enginner jobs in the US (using 30 urls)
url = 'https://www.glassdoor.com/Job/us-software-engineer-jobs-SRCH_IL.0,2_IN1_KO3,20.htm'
url = "https://www.glassdoor.com/Job/us-software-engineer-jobs-SRCH_IL.0,2_IN1_KO3,20_IP2.htm"
:
url = "https://www.glassdoor.com/Job/us-software-engineer-jobs-SRCH_IL.0,2_IN1_KO3,20_IP30.htm"

In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
from datetime import datetime
import pandas as pd

In [74]:
def get_jobs(verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="chromedriver", options=options)
    driver.set_window_size(1120, 1000)
    
    #url = 'https://www.glassdoor.com/Job/us-software-engineer-jobs-SRCH_IL.0,2_IN1_KO3,20.htm'
    #url = "https://www.glassdoor.com/Job/us-software-engineer-jobs-SRCH_IL.0,2_IN1_KO3,20_IP30.htm"

    #url = 'https://www.glassdoor.com/Job/us-data-scientist-jobs-SRCH_IL.0,2_IN1_KO3,17.htm'
    url = 'https://www.glassdoor.com/Job/us-data-scientist-jobs-SRCH_IL.0,2_IN1_KO3,17_IP30.htm'
    
    driver.get(url)
    jobs = []

    #Let the page load. Change this number based on your internet speed.
    #Or, wait until the webpage is loaded, instead of hardcoding it.
    time.sleep(4)

    #Test for the "Sign Up" prompt and get rid of it.
    try:
        driver.find_element_by_class_name("selected").click()
    except ElementClickInterceptedException:
        pass

    time.sleep(3)

    try:
        driver.find_element_by_class_name("ModalStyle__xBtn___29PT9").click()  #clicking to the X.
    except NoSuchElementException:
        pass

    timestamp = datetime.now().isoformat()

    #Going through each job in this page
    job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
    time.sleep(.5)
    for job_button in job_buttons:  

        print("Progress: {}".format("" + str(len(jobs))))

        job_button.click()  #You might 
        time.sleep(1)
        collected_successfully = False

        while not collected_successfully:
            try:
                company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                location = driver.find_element_by_xpath('.//div[@class="location"]').text
                job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                collected_successfully = True
            except:
                time.sleep(5)

        time.sleep(0.1)
        try:
            salary_estimate = driver.find_element_by_xpath('.//span[@class="gray small salary"]').text
        except NoSuchElementException:
            salary_estimate = -1 #You need to set a "not found value. It's important."

        try:
            rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
        except NoSuchElementException:
            rating = -1 #You need to set a "not found value. It's important."

        #Printing for debugging
        if verbose:
            print("Job Title: {}".format(job_title))
            print("Salary Estimate: {}".format(salary_estimate))
            print("Job Description: {}".format(job_description[:500]))
            print("Rating: {}".format(rating))
            print("Company Name: {}".format(company_name))
            print("Location: {}".format(location))

        #Going to the Company tab...
        #clicking on this:
        #<div class="tab" data-tab-type="overview"><span>Company</span></div>
        time.sleep(0.5)
        try:
            driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

            try:
                #<div class="infoEntity">
                #    <label>Headquarters</label>
                #    <span class="value">San Francisco, CA</span>
                #</div>
                headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
            except NoSuchElementException:
                headquarters = -1

            try:
                size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
            except NoSuchElementException:
                size = -1

            try:
                founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
            except NoSuchElementException:
                founded = -1

            try:
                type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
            except NoSuchElementException:
                type_of_ownership = -1

            try:
                industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
            except NoSuchElementException:
                industry = -1

            try:
                sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
            except NoSuchElementException:
                sector = -1

            try:
                revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
            except NoSuchElementException:
                revenue = -1

            try:
                competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
            except NoSuchElementException:
                competitors = -1

        except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
            headquarters = -1
            size = -1
            founded = -1
            type_of_ownership = -1
            industry = -1
            sector = -1
            revenue = -1
            competitors = -1


        if verbose:
            print("Headquarters: {}".format(headquarters))
            print("Size: {}".format(size))
            print("Founded: {}".format(founded))
            print("Type of Ownership: {}".format(type_of_ownership))
            print("Industry: {}".format(industry))
            print("Sector: {}".format(sector))
            print("Revenue: {}".format(revenue))
            print("Competitors: {}".format(competitors))
            print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

        jobs.append({"Job Title" : job_title,
        "Salary Estimate" : salary_estimate,
        "Job Description" : job_description,
        "Rating" : rating,
        "Company Name" : company_name,
        "Location" : location,
        "Headquarters" : headquarters,
        "Size" : size,
        "Founded" : founded,
        "Type of ownership" : type_of_ownership,
        "Industry" : industry,
        "Sector" : sector,
        "Revenue" : revenue,
        "Competitors" : competitors,
        "timestamp" : timestamp})
        #add job to jobs


    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [75]:
#This line will open a new chrome window and start the scraping.
df = get_jobs(False)
df

Progress: 0
Progress: 1
Progress: 2
Progress: 3
Progress: 4
Progress: 5
Progress: 6
Progress: 7
Progress: 8
Progress: 9
Progress: 10
Progress: 11
Progress: 12
Progress: 13
Progress: 14
Progress: 15
Progress: 16
Progress: 17
Progress: 18
Progress: 19
Progress: 20
Progress: 21
Progress: 22
Progress: 23
Progress: 24
Progress: 25
Progress: 26
Progress: 27
Progress: 28
Progress: 29
Progress: 30
Progress: 31


,Company Name,Competitors,Founded,Headquarters,Industry,Job Description,Job Title,Location,Rating,Revenue,Salary Estimate,Sector,Size,Type of ownership,timestamp
0,TransUnion\n3.9,-1,1968,"Chicago, IL",Financial Analytics & Research,What We'll Bring:\n\nWhat We'll Bring:\n\nAt T...,Insurance Data Scientist,"Atlanta, GA",3.9,$1 to $2 billion (USD),-1,Finance,5001 to 10000 employees,Company - Public,2020-04-01T07:58:08.095928
1,MITRE\n3.2,"Battelle, General Atomics, SAIC",1958,"Bedford, MA",Federal Agencies,MITRE\nis a trusted operator of federally fund...,Jr. Data Scientist,"McLean, VA",3.2,$1 to $2 billion (USD),-1,Government,5001 to 10000 employees,Nonprofit Organization,2020-04-01T07:58:08.095928
2,MITRE\n3.2,"Battelle, General Atomics, SAIC",1958,"Bedford, MA",Federal Agencies,MITRE is different from most technology compan...,Machine Learning - Computer Scientist,"McLean, VA",3.2,$1 to $2 billion (USD),-1,Government,5001 to 10000 employees,Nonprofit Organization,2020-04-01T07:58:08.095928
3,Pfizer\n4.0,-1,1849,"New York, NY",Biotech & Pharmaceuticals,ROLE SUMMARY\n\nResponsible for the oversight ...,"Senior Principal Scientist, DMPK","Cambridge, MA",4.0,$10+ billion (USD),-1,Biotech & Pharmaceuticals,10000+ employees,Company - Public,2020-04-01T07:58:08.095928
4,The Syllogisteks Company\n4.3,-1,1992,"Chesterfield, MO",Staffing & Outsourcing,Must be able to work as our W2 employee *Senio...,"Full Stack Developer (Java, Scala, Talend, Dat...","Creve Coeur, MO",4.3,$10 to $25 million (USD),-1,Business Services,Unknown,Company - Private,2020-04-01T07:58:08.095928
5,Raytheon\n3.7,-1,1922,"Waltham, MA",Aerospace & Defense,"Are you an experienced, passionate pioneer in ...",Software Engineer Staff Scientist: Human Langu...,"Cambridge, MD",3.7,$10+ billion (USD),-1,Aerospace & Defense,10000+ employees,Company - Public,2020-04-01T07:58:08.095928
6,"PTC Therapeutics, Inc.\n3.2",-1,1998,"South Plainfield, NJ",Biotech & Pharmaceuticals,Job Description Summary:\nJob Description:\n\n...,"PhD Scientist, Enzymology","Mountain View, CA",3.2,$100 to $500 million (USD),-1,Biotech & Pharmaceuticals,501 to 1000 employees,Company - Public,2020-04-01T07:58:08.095928
7,AstraZeneca\n3.9,"Roche, GlaxoSmithKline, Novartis",1913,"Cambridge, United Kingdom",Biotech & Pharmaceuticals,"AstraZeneca is a global, innovation-driven bio...","Sr Scientist, Purification Process Sciences","Gaithersburg, MD",3.9,$10+ billion (USD),-1,Biotech & Pharmaceuticals,10000+ employees,Company - Public,2020-04-01T07:58:08.095928
8,Druva\n2.8,"Rubrik, Commvault, Cohesity",2007,"Sunnyvale, CA",Enterprise Software & Network Solutions,Druva delivers data protection and management ...,Business Intelligence Analyst - GTM,"Sunnyvale, CA",2.8,$100 to $500 million (USD),-1,Information Technology,501 to 1000 employees,Company - Private,2020-04-01T07:58:08.095928
9,Precision Biosciences\n5.0,-1,2006,"Durham, NC",Biotech & Pharmaceuticals,"Summary\n\nThe Process Development Scientist, ...","Scientist, mRNA Process Development","Durham, NC",5.0,Unknown / Non-Applicable,-1,Biotech & Pharmaceuticals,201 to 500 employees,Company - Public,2020-04-01T07:58:08.095928


In [76]:
df.to_csv(r'dsdf30.csv', index = False, header=False)

In [ ]:
#df.to_csv(r'dsdf1.csv', index = False, header=True)